In [1]:
import os
import gc
import warnings

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import missingno as msno
import scipy.stats  as ss

from statsmodels.api import ProbPlot


warnings.filterwarnings(action = 'ignore')
pd.set_option('display.max_rows', None)

In [2]:
train_df = pd.read_csv("../dataset/Cleaned_train.csv")

In [3]:
# 类别、离散特征
cate_col_list = ['ProductCD','P_emaildomain','R_emaildomain','DeviceType','DeviceInfo','addr1','addr2']

#循环加入特征
for i in range(1,7):
    cate_col_list.append('card' + str(i))
for i in range(1 ,10):
    cate_col_list.append('M' + str(i))
for i in range(12 ,39):
    cate_col_list.append('id_' + str(i // 10) + str(i % 10))
#连续型特征
num_col_list = [x for x in train_df.columns if x not in cate_col_list]
num_col_list.remove('isFraud')

In [13]:
train_df['TransactionAmt'].value_counts()

999999.000    101553
59.000         30582
117.000        28933
107.950        23954
57.950         23600
100.000        20362
50.000         17009
49.000         15990
39.000         11095
29.000         10683
47.950          9655
35.950          8909
150.000         8356
171.000         7682
34.000          7652
30.950          7649
25.000          7500
77.000          7389
25.950          7294
67.950          5403
75.000          5107
159.950         4716
97.000          4678
92.000          4280
58.950          3798
30.000          3571
108.950         3564
82.950          3262
108.500         3219
15.000          3021
40.000          2955
54.500          2783
20.000          2710
87.000          2454
87.950          2443
44.000          2360
141.000         2321
125.000         2287
54.000          2219
77.950          1971
48.950          1790
42.950          1718
36.950          1717
146.000         1597
24.000          1465
44.500          1462
24.500          1355
97.950       

In [5]:
print(cate_col_list, end = '')

['ProductCD', 'P_emaildomain', 'R_emaildomain', 'DeviceType', 'DeviceInfo', 'addr1', 'addr2', 'card1', 'card2', 'card3', 'card4', 'card5', 'card6', 'M1', 'M2', 'M3', 'M4', 'M5', 'M6', 'M7', 'M8', 'M9', 'id_12', 'id_13', 'id_14', 'id_15', 'id_16', 'id_17', 'id_18', 'id_19', 'id_20', 'id_21', 'id_22', 'id_23', 'id_24', 'id_25', 'id_26', 'id_27', 'id_28', 'id_29', 'id_30', 'id_31', 'id_32', 'id_33', 'id_34', 'id_35', 'id_36', 'id_37', 'id_38']

In [6]:
print(num_col_list, end = '')

['TransactionID', 'TransactionDT', 'TransactionAmt', 'dist1', 'dist2', 'C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'C7', 'C8', 'C9', 'C10', 'C11', 'C12', 'C13', 'C14', 'D1', 'D2', 'D3', 'D4', 'D5', 'D6', 'D7', 'D8', 'D9', 'D10', 'D11', 'D12', 'D13', 'D14', 'D15', 'V1', 'V2', 'V3', 'V4', 'V5', 'V6', 'V7', 'V8', 'V9', 'V10', 'V11', 'V12', 'V13', 'V14', 'V15', 'V16', 'V17', 'V18', 'V19', 'V20', 'V21', 'V22', 'V23', 'V24', 'V25', 'V26', 'V27', 'V28', 'V29', 'V30', 'V31', 'V32', 'V33', 'V34', 'V35', 'V36', 'V37', 'V38', 'V39', 'V40', 'V41', 'V42', 'V43', 'V44', 'V45', 'V46', 'V47', 'V48', 'V49', 'V50', 'V51', 'V52', 'V53', 'V54', 'V55', 'V56', 'V57', 'V58', 'V59', 'V60', 'V61', 'V62', 'V63', 'V64', 'V65', 'V66', 'V67', 'V68', 'V69', 'V70', 'V71', 'V72', 'V73', 'V74', 'V75', 'V76', 'V77', 'V78', 'V79', 'V80', 'V81', 'V82', 'V83', 'V84', 'V85', 'V86', 'V87', 'V88', 'V89', 'V90', 'V91', 'V92', 'V93', 'V94', 'V95', 'V96', 'V97', 'V98', 'V99', 'V100', 'V101', 'V102', 'V103', 'V104', 'V105', 'V106', 'V10

# Feature Engineering

- 目的并不是要做这个项目到极致，因为采用的是集成学习这种复杂模型，暂时不必采用分箱处理
- 采用树模型，暂时也不必做数据缩放处理

## Feature Encoding